## About
This notebook calculates the root mean-squared 1-68 distance for the NTD either unbound or in the presence of RNA.

In [5]:
from soursop.sstrajectory import SSTrajectory
import mdtraj as md
import numpy as np

In [7]:
# define the directory where NTD-RBD simulations are found
prefix = '/work/alstonj/2022/Binding/FINAL/NTDRBD/'

e2e_final = []
e2e_std = []

e2e_final_raw = []
e2e_std_raw = []

RNA_lengths = [0,10, 12, 15, 17, 20, 25, 30, 35, 40]
n_reps=3

# cycle over RNA lengths
for RNA_len in RNA_lengths:
    
    e2e_local = []
    e2e_local_raw = []
    
    for rep in range(1,n_reps+1):
        
        
        # if RNA length is 0 select all frames
        if RNA_len == 0:
            xtcfilename = prefix + f'{RNA_len}/{rep}/equilibrated.xtc'
            pdbfilename = prefix + f'{RNA_len}/{rep}/equilibrated.pdb'   
            
        # if RNA length is greater than 0 we only analyse the bound-state frames
        else:
            xtcfilename = prefix + f'{RNA_len}/{rep}/bound.xtc'
            pdbfilename = prefix + f'{RNA_len}/{rep}/equilibrated.pdb'   
            
        T = SSTrajectory(xtcfilename,pdbfilename, extra_valid_residue_names=['RPU'])
        
        e2e = T.proteinTrajectoryList[0].get_inter_residue_COM_distance(R1=1, R2=68)

        e2e = e2e/10
        
        # calculate root-mean square distance  (i..e rms end-to-end distance)
        e2e_local.append(np.sqrt(np.mean(np.power(e2e,2))))
        e2e_local_raw.append(np.mean(e2e))
        
        
    e2e_final.append(np.mean(e2e_local))
    e2e_std.append(np.std(e2e_local))
    
    e2e_final_raw.append(np.mean(e2e_local_raw))
    e2e_std_raw.append(np.std(e2e_local_raw))

In [8]:
with open('rmsd_ntd_1_68.csv','w') as fh:
    fh.write('#RNA length, RMS 1-68, std RMS 1-68, <r> 1-68, std(<r>) 1-68\n')
    for idx in range(len(e2e_std)):
        fh.write(f'{RNA_lengths[idx]}, {e2e_final[idx]}, {e2e_std[idx]}, {e2e_final_raw[idx]}, {e2e_std_raw[idx]}\n')